**Author**

Shamim, 
Remote Backend Engineer at Short Circuit Science<br>
London, UK <br>
[GitHub](https://github.com/anamulislamshamim) [LinkedIn](https://www.linkedin.com/in/anamul-islam-shamim/)

**Context Manager**

A context manager is a `Python` construct that sets up a resources, makes it available, and then ensures it's cleaned up properly - no matter what happens (even if an error occurs).

It is what powers the `with` statement.


**When should we use it?**

We use context managers whenever we deal with resources that need setup and teardown, such as:
* 1. Opening/Closing files
* 2. Acquiring/releasing locks (threading, multiprocessing)
* 3. Managing DB connections or transactions
* 4. Network connections, sockets
* 5. Temporary changes to environment/config

**Implementation**

1. Class Based: Using with + __enter__, __exit__ dander functions

In [10]:
class MyContext:
    def __enter__(self):
        print("Entering context...")
        return "Shamim Resources"
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        print("Exit context")
        if exc_type:
            print(f"Error: {exc_val}")
        
        return False # False to propagate exception to the caller
        

In [11]:
try:
    with MyContext() as shamim_context:
        print(f"Allocate {shamim_context}...")
        raise ValueError("Shamim error")
except ValueError as e:
    print(f"Caught exception: {e}")

Entering context...
Allocate Shamim Resources...
Exit context
Error: Shamim error
Caught exception: Shamim error


2. Using contextlib.contextmanager (Cleaner)

In [13]:
from contextlib import contextmanager


@contextmanager
def my_context():
    print("Entering context...")
    yield "Resources_Shamim"
    print("Existing context")

with my_context() as res:
    print("Allocating", res)

Entering context...
Allocating Resources_Shamim
Existing context


**Atomic Transaction using Context Manager**

In [31]:
from contextlib import contextmanager
import sqlite3


@contextmanager
def db_transaction(conn, commit=True):
    cursor = conn.cursor()
    try:
        yield cursor
        if commit:
            conn.commit()
        print("Transaction completed successfully")
    except Exception as e:
        conn.rollback()
        print(f"Transaction rollback on error: {e}")
    finally:
        cursor.close()

conn = sqlite3.connect('test.db')

# try:
#     with db_transaction(conn) as cur:
#         # Note: We use IF NOT EXISTS to prevent errors if the table is already present
#         cur.execute("""
#             CREATE TABLE IF NOT EXISTS users (
#                 id INTEGER PRIMARY KEY,
#                 name TEXT NOT NULL,
#                 age INTEGER,
#                 email TEXT UNIQUE
#             );
#         """)
#         # We set commit=False here just to show we can, but by default it commits.
#         print("SQL: CREATE TABLE executed.")
# except Exception as e:
#     # This try/except handles the unlikely case of a failure during table creation
#     print(f"Failed to create table: {e}")

with db_transaction(conn) as cur:
    cur.execute("INSERT INTO users(name, age, email) VALUES(?, ?, ?)", ('Shamim', 30, 'shamim@example.com'))
    data = cur.execute("SELECT * FROM users").fetchall()
    print("data", data)

Transaction rollback on error: UNIQUE constraint failed: users.email


In [32]:
with db_transaction(conn) as cur:
    data = cur.execute("SELECT * FROM users").fetchall()
    print("data", data)

data [(1, 'Shamim', 30, 'shamim@example.com'), (2, 'Shamim', 30, 'tamim@example.com')]
Transaction completed successfully


**Implement Execution Time Context Manager**

1.Using Class, __enter__, __exit__ dander methods

In [33]:
import time 


class Timer:
    def __enter__(self):
        self.start = time.time() 
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.end = time.time()
        print(f"Execution time: {self.end - self.start:.5f} seconds")
        return False # to propagate exception to the caller

In [34]:
with Timer():
    total = 0
    for i in range(10_000_000):
        total += i

Execution time: 2.00366 seconds


2. Using contextmanager decorator

In [35]:
from contextlib import contextmanager


@contextmanager
def my_timer():
    start = time.time()

    try:
        yield
    except Exception as e:
        raise e
    finally:
        end = time.time()
        print(f"Executed: {end-start:.5f} seconds.")

In [36]:
with my_timer():
    total = 0
    for i in range(10_000_000):
        total += i

Executed: 2.42180 seconds.


**Real-World Relevance**

* Profiling API endpoints in Django/FastAPI.
* Checking slow database queries.
* Benchmarking algorithms.
* Identifying performance bottlenecks in production code.